In [4]:
import polars as pl
import matplotlib.pyplot as plt

from common.constants.column_types import CPZP_SCHEMA, OZP_SCHEMA
from common.constants.column_names import SHARED_COLUMNS, OZP_COLUMNS
from datetime import datetime

pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(6)


def read_preskladane_data(file_path: str, schema: pl.Schema) -> pl.DataFrame:
    return pl.read_csv(
        file_path,
        null_values=["NA", ""],
        schema=schema,
    )


cpzp_df = read_preskladane_data("./DATACON_data/CPZP_preskladane.csv", CPZP_SCHEMA)
ozp_df = read_preskladane_data("./DATACON_data/OZP_preskladane.csv", OZP_SCHEMA)
print(f"CPZP: {cpzp_df.schema}")

CPZP: Schema({'Id_pojistence': Float64, 'Pohlavi': Enum(categories=['M', 'Z']), 'Rok_narozeni': Int64, 'Mesic_narozeni': Int64, 'Posledni_zahajeni_pojisteni': Date, 'Posledni_ukonceni_pojisteni': Date, 'Rok_umrti': Int64, 'Mesic_umrti': Int64, 'Typ_udalosti': Enum(categories=['předpis', 'vakcinace']), 'Kod_udalosti': String, 'Detail_udalosti': String, 'Pocet_baleni': Float64, 'Datum_udalosti': Date, 'Specializace': String, 'Datum_umrti': Date, 'léková_forma_zkr': String, 'ATC_skupina': String, 'síla': String, 'doplněk_názvu': String, 'léková_forma': String, 'léčivé_látky': String, 'Equiv_sloucenina': Enum(categories=['TRIAMCINOLON', 'FLUDROKORTISON', 'METHYLPREDNISOLON', 'DEXAMETHASON', 'PREDNISON', 'BETAMETHASON', 'HYDROKORTISON']), 'Prednison_equiv': Float64, 'Pocet_v_baleni': Float64, 'pololeti': Int64, 'rok_zahajeni': Int64, 'poradi': Int64, 'pocet_vakcinaci': Int64, 'ockovany': Enum(categories=['1', '0']), 'pocet_predpisu': Int64})


In [16]:
def precise_age_expr():
    return (pl.col("Datum_udalosti").dt.year() - pl.col("Rok_narozeni")) - (
        (pl.col("Datum_udalosti").dt.month() < pl.col("Mesic_narozeni"))
        | (
            (pl.col("Datum_udalosti").dt.month() == pl.col("Mesic_narozeni"))
            & (pl.col("Datum_udalosti").dt.day() < 1)  # birthdays assumed day=1
        )
    ).cast(pl.Int8)


def curren_age_expr():
    # Use death date if present, otherwise today's date
    event_date = (
        pl.when(pl.col("Datum_umrti").is_not_null())
        .then(pl.col("Datum_umrti"))
        .otherwise(pl.lit(datetime.now().date()))
    )

    # Birthday-aware age calculation
    age = (event_date.dt.year() - pl.col("Rok_narozeni")) - (
        (event_date.dt.month() < pl.col("Mesic_narozeni"))
        | (
            (event_date.dt.month() == pl.col("Mesic_narozeni"))
            & (event_date.dt.day() < 1)  # assumed birthday day=1
        )
    ).cast(pl.Int8)

    return age


age = precise_age_expr()
current_age = curren_age_expr()

In [17]:
df = cpzp_df

# 1. Počet osob s očkováním
vax_persons_count = (
    df.filter(pl.col("ockovany") == "1").select("Id_pojistence").n_unique()
)
print(f"Počet osob s očkováním: {vax_persons_count}")

# 2. Počet osob bez očkování a bez předpisů
no_vax_no_prescriptions_count = (
    df.filter((pl.col("ockovany") == "0") & (pl.col("pocet_predpisu") == 0))
    .select("Id_pojistence")
    .n_unique()
)
print(f"Počet osob bez očkování a bez předpisů: {no_vax_no_prescriptions_count}")

# 3. Počet předpisů pro osoby 30–50 let
# -> Věk v době události = rok události - rok narození
prescriptions_for_30_50_count = df.filter(
    (pl.col("Typ_udalosti") == "předpis") & (age >= 30) & (age < 50)
).height
print(f"Počet předpisů pro osoby 30-50 let: {prescriptions_for_30_50_count}")

# 4. Počet očkovaných osob pro analýzu 30–50 let bez úmrtí
analysis_included_30_50_vax_count = (
    df.filter(
        (pl.col("ockovany") == "1")
        & (current_age >= 30)
        & (current_age < 50)
        & pl.col("Datum_umrti").is_null()
    )
    .select("Id_pojistence")
    .n_unique()
)
print(
    f"Počet očkovaných osob pro analýzu 30-50 let: {analysis_included_30_50_vax_count}"
)

# 5. Počet předpisů pro očkované osoby 30–50 let v letech 2020–2023
prescriptions_for_30_50_vax_2020_2023 = df.filter(
    (pl.col("ockovany") == "1")
    & (pl.col("Typ_udalosti") == "předpis")
    & (age >= 30)
    & (age < 50)
    & (pl.col("Datum_udalosti") >= datetime(2020, 1, 1).date())
    & (pl.col("Datum_udalosti") <= datetime(2023, 1, 1).date())
).height
print(
    f"Počet předpisů pro očkované osoby 30-50 let v letech 2020-2023: {prescriptions_for_30_50_vax_2020_2023}"
)

# 6. Počet prvopředpisů v letech 2020–2023
#   -> Najdeme první předpis pro každého pojištěnce a zkontrolujeme datum
first_prescriptions_count = (
    df.filter(pl.col("Typ_udalosti") == "předpis")
    .group_by("Id_pojistence")
    .agg(pl.col("Datum_udalosti").min().alias("first_prescription_date"))
    .filter(
        (pl.col("first_prescription_date") >= datetime(2020, 1, 1).date())
        & (pl.col("first_prescription_date") <= datetime(2023, 1, 1).date())
    )
    .height
)
print(f"Počet prvopředpisů v letech 2020-2023: {first_prescriptions_count}")

Počet osob s očkováním: 765972
Počet osob bez očkování a bez předpisů: 630500
Počet předpisů pro osoby 30-50 let: 735528
Počet očkovaných osob pro analýzu 30-50 let: 225939
Počet předpisů pro očkované osoby 30-50 let v letech 2020-2023: 180884
Počet prvopředpisů v letech 2020-2023: 124820


In [2]:
vax_or_prescription = cpzp_df.filter(
    (((pl.col("Typ_udalosti") == "vakcinace") | (pl.col("Typ_udalosti") == "předpis")))
)

non_vax_and_non_prescription = cpzp_df.filter(
    ~(((pl.col("Typ_udalosti") == "vakcinace") | (pl.col("Typ_udalosti") == "předpis")))
    | (pl.col("Typ_udalosti").is_null())
)

print(len(cpzp_df))
print(len(vax_or_prescription))
print(len(non_vax_and_non_prescription))
assert len(vax_or_prescription) + len(non_vax_and_non_prescription) == len(cpzp_df)


vax_or_prescription = ozp_df.filter(
    (((pl.col("Typ_udalosti") == "vakcinace") | (pl.col("Typ_udalosti") == "předpis")))
)

non_vax_and_non_prescription = ozp_df.filter(
    ~(((pl.col("Typ_udalosti") == "vakcinace") | (pl.col("Typ_udalosti") == "předpis")))
    | (pl.col("Typ_udalosti").is_null())
)

print(len(ozp_df))
print(len(vax_or_prescription))
print(len(non_vax_and_non_prescription))
assert len(vax_or_prescription) + len(non_vax_and_non_prescription) == len(ozp_df)

5478462
4847962
630500
3642969
2871013
771956
